# Model Experimentation

In [1]:
# Change directory to keep paths consistent
%cd /Users/brk/projects/masters/SU/ergo/src

/Users/brk/projects/masters/SU/ergo/src


In [2]:
# minimise me
%load_ext autoreload
%autoreload 2
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import ipywidgets as widgets
import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import numpy as np
import models
import vis
import common
import read
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
import sklearn
import tqdm
import logging as l
import tqdm
import yaml
import glob
from matplotlib.colors import LogNorm
import re
from sklearn.metrics import classification_report

## Read in the training data

In [3]:
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

## Regular FFNN model

In [ ]:
keras.backend.clear_session()
num_layers = 1
nodes_per_layer = [187]
model_type = "FFNN"
num_gesture_classes = 51
preprocessing: models.PreprocessingConfig = {
    'seed': 42 + 0,
    'n_timesteps': 20,
    'max_obs_per_class': 200 if model_type == "HMM" else None,
    'gesture_allowlist': list(range(num_gesture_classes)),
    'num_gesture_classes': num_gesture_classes,
    'rep_num': 0
}


config: models.ConfigDict = {
    "model_type": "FFNN",
    "preprocessing": preprocessing,
    "nn": {
        "epochs": 30,
        "batch_size": 243,
        "learning_rate": 0.00028699308114633,
        "optimizer": "adam",
    },
    "ffnn": {
        "nodes_per_layer": nodes_per_layer,
        "l2_coefficient": 4.339192561757433e-05,
        "dropout_rate": 0.09057350661309399,
    },
    "cusum": None,
    "lstm": None,
    "hmm": None,
}
# pprint(config)
clf = models.FFNNClassifier(config=config)
print("Fitting model")
start = datetime.datetime.now()
                                                                                       
clf.fit(
    X_trn,
    y_trn,
    dt_trn,
    validation_data=(X_val, y_val, dt_val),
    verbose=True,
    callbacks=[
        models.DisplayConfMat(
            validation_data=(X_val, y_val, dt_val),
            conf_mat=False,
#             fig_path=f'saved_models/{study_name}/trial_{trial.number}.png',
        ),
    ]
)
finsh = datetime.datetime.now()
# score = calc_metrics(trial, start, finsh, clf)


### Calculate metrics for the FFNN model

In [ ]:
model_type = clf.config['model_type']
X_val = clf.validation_data[0]
y_val = clf.validation_data[1]
duration = finsh - start
duration_ms = duration.seconds * 1000 + duration.microseconds / 1000
print("duration_ms", duration_ms)
                                                                         
if model_type == "FFNN":
    print("val_loss", clf.history.history["val_loss"][-1])
    print("trn_loss", clf.history.history["loss"][-1])
                                                                         
jsonl_path = f"../saved_models/results_{model_type.lower()}_optuna.jsonl"
model_dir = clf.write_as_jsonl(jsonl_path)
if model_type == "FFNN":
    print(model_dir)
    clf.dump(model_dir)
                                                                         
print("model_dir", model_dir)
                                                                         
y_pred = clf.predict(X_val)
clf_report = pd.json_normalize(sklearn.metrics.classification_report(
    y_val.astype(int),
    y_pred.astype(int),
    output_dict=True,
    zero_division=0,
))
print(sklearn.metrics.classification_report(
    y_val.astype(int),
    y_pred.astype(int),
    output_dict=False,
    zero_division=0,
))
                                                                         
print("val.macro avg.f1-score", clf_report['macro avg.f1-score'].values[0])
print("val.macro avg.precision", clf_report['macro avg.precision'].values[0])
print("val.macro avg.recall", clf_report['macro avg.recall'].values[0])
print (clf_report['macro avg.f1-score'].values[0])


## Experiment with a two-stage FFNN model

In [ ]:
majority_config: models.ConfigDict = {
    "model_type": "FFNN",
    "preprocessing": {
        'seed': 42 + 0,
        'n_timesteps': 20,
        'max_obs_per_class': None,
        'gesture_allowlist': [0, 1],
        'num_gesture_classes': 2,
        'rep_num': 0
    },
    "nn": {
        "epochs": 15,
        "batch_size": 256,
        "learning_rate": 0.0005,
        "optimizer": "adam",
    },
    "ffnn": {
        "nodes_per_layer": [100, 100],
        "l2_coefficient": 4.3e-05,
        "dropout_rate": 0.09,
    },
    "cusum": None,
    "lstm": None,
    "hmm": None,
}

minority_config: models.ConfigDict = {
    "model_type": "FFNN",
    "preprocessing": {
        'seed': 42 + 0,
        'n_timesteps': 20,
        'max_obs_per_class': None,
        'gesture_allowlist': list(range(50)),
        'num_gesture_classes': 50,
        'rep_num': 0
    },
    "nn": {
        "epochs": 30,
        "batch_size": 256,
        "learning_rate": 0.0005,
        "optimizer": "adam",
    },
    "ffnn": {
        "nodes_per_layer": [100, 100],
        "l2_coefficient": 4.3e-05,
        "dropout_rate": 0.09,
    },
    "cusum": None,
    "lstm": None,
    "hmm": None,
}

meta_clf = models.MetaClassifier(
    majority_config=majority_config,
    minority_config=minority_config,
)
meta_clf.fit(
    X_trn,
    y_trn,
    dt_trn,
    validation_data=(X_val, y_val, dt_val),
    verbose=True,
    callbacks=[
        models.DisplayConfMat(
            validation_data=(X_val, y_val, dt_val),
        ),
    ]
)

### Calculate metrics for the two-stage FFNN model

In [ ]:
y_pred = meta_clf.predict(X_val)

conf_mat = tf.math.confusion_matrix(y_val, y_pred).numpy()
print(conf_mat)
vis.conf_mat(conf_mat);
plt.show()
print(sklearn.metrics.classification_report(
    y_val.astype(int),
    y_pred.astype(int),
    zero_division=0,
))


## Plot the PCA of the data, along with model predictions

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_reshaped = X_val.reshape((X_val.shape[0], -1))
X_tfrm = pca.fit_transform(X_reshaped[y_val != 50])

argsort = np.argsort(y_val[y_val != 50])

hues = np.array([ "0°", "45°", "90°", "135°", "180°" ])
styles = np.array([ "L1", "L2", "L3", "L4", "L5", "R5", "R4", "R3", "R2", "R1" ])

fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=300)

sns.scatterplot(
    x=X_tfrm[:, 0][argsort][(y_val != y_pred)[y_val != 50]],
    y=X_tfrm[:, 1][argsort][(y_val != y_pred)[y_val != 50]],
    s=60,
    ax=ax,
    alpha=.25,
    edgecolor='red',
    color='red',
    linewidth=.1,
)

sns.scatterplot(
    x=X_tfrm[:, 0][argsort],
    y=X_tfrm[:, 1][argsort],
    hue=hues[(y_val[y_val != 50][argsort] // 10)],
    style=styles[(y_val[y_val != 50][argsort] % 10)],
    s=30,
    ax=ax
)

plt.title("PCA plot of the validation data\nExcluding gesture 50\nIncorrect predictions circled")
# TODO maybe have an overlay of the datapoints that the model gets wrong?

## Experiment with SVM (22 minute training time)

In [ ]:
clf = models.SVMClassifier(config={
    "model_type": "SVM",
    "preprocessing": {
        'seed': 42 + 0,
        'n_timesteps': 20,
        'max_obs_per_class': None,
        'gesture_allowlist': list(range(51)),
        'num_gesture_classes': 51,
        'rep_num': 0
    },
    "svm": {
        "c": 1.0,
        "class_weight": None,
        "max_iter": 40,
    },
    "nn": None,
    "ffnn": None,
    "cusum": None,
    "lstm": None,
    "hmm": None,
})

clf.fit(
    X_trn,
    y_trn,
    dt_trn,
    validation_data=(X_val, y_val, dt_val),
    verbose=True,
)

### Calculate metrics for the SVM model

In [ ]:
y_pred = clf.predict(X_val)

conf_mat = tf.math.confusion_matrix(y_val, y_pred).numpy()
print(conf_mat)
conf_mat[-1,-1] = 0
vis.conf_mat(conf_mat, norm=None);
plt.show()
print(sklearn.metrics.classification_report(
    y_val.astype(int),
    y_pred.astype(int),
    zero_division=0,
))